<a href="https://colab.research.google.com/github/AkbarFauzy/AI-Enabled_IOT-AccelometerBreathCirculation/blob/main/UTS_IOT___1301194133.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.signal import butter, filtfilt
import warnings
warnings.filterwarnings("ignore")

plt.rcParams['figure.figsize'] = [15,3]
plt.rcParams.update({'font.size' : 18})

In [ ]:
# Fungsi Plotting
def fig_from_df(df):
    fig = go.Figure()
    for col in df.columns:
      fig.add_trace(go.Scatter(x=df.index,y=df[col],name=col))
    return fig

In [ ]:
# Import Data
data = pd.read_excel("https://github.com/AkbarFauzy/Breath_Signal_Dataset/blob/main/100_hz_2021-11-15_21-03-49.xls?raw=true")
# Menghilangkan Spasi pada nama kolom
data.columns = data.columns.str.replace(' ','_')

In [ ]:
# Set index
data = data.set_index("Time_(s)")

In [ ]:
Az= data["Linear_Acceleration_z_(m/s^2)"]
Ay= data["Linear_Acceleration_y_(m/s^2)"]
Ax= data["Linear_Acceleration_x_(m/s^2)"]

# Mengenolkan sinyal pada sumbu Y
Az = Az-(sum(Az)/len(Az))
Ay = Ay-(sum(Ay)/len(Ay))
Ax = Ax-(sum(Ax)/len(Ax))

data["Linear_Acceleration_z_(m/s^2)"] = Az
data["Linear_Acceleration_y_(m/s^2)"] = Ay
data["Linear_Acceleration_x_(m/s^2)"] = Ax

# Plot Raw Sinyal
fig = fig_from_df(data)
fig.write_html('time-history.html',full_html=False,include_plotlyjs='cdn')
fig.update_layout(
    title = "Raw Signal",
    xaxis_title = "Time (s)",
    yaxis_title = "Acc (m/s^2)"
)
fig.show()


In [ ]:
# Mendapatkan sudut inklinasi sudut Y terhadap sudut Z
data_inklinasi = pd.DataFrame(np.arctan2(data["Linear_Acceleration_y_(m/s^2)"], data["Linear_Acceleration_z_(m/s^2)"]))

In [ ]:
# Plotting Data Inklinasi
fig = fig_from_df(data_inklinasi)
fig.write_html('time-history.html',full_html=False,include_plotlyjs='cdn')
fig.update_layout(
    title = "Raw Data",
    xaxis_title = "Time (s)",
    yaxis_title = "Acc (m/s^2)"
)
fig.update_layout(
    title = "Raw Signal Inklinasi",
    xaxis_title = "Time (s)",
    yaxis_title = "Degree"
)
fig.show()


In [ ]:
# Fungsi Lowpass
def lowpass(data, cutoff,fs,order):
  normal_cutoff = cutoff/(0.5*fs)
  b, a = butter(order, normal_cutoff, btype='low')
  y = filtfilt(b,a,data)
  return y

In [ ]:
fs = 100 #Frekuensi Sample
cutoff = 0.8 #Cutoff
nyq = 0.5 * fs # Nyquist frequency
order = 2 #Order

In [ ]:
# lowpass pada sudut Y
y_filtered = lowpass(data["Linear_Acceleration_y_(m/s^2)"], cutoff,fs,order)
# Menambahkan hasil lowpass pada dataframe data
data['y_filtered'] = y_filtered

#Plotting hasil filter sinyal Raw
fig = go.Figure()
fig.add_trace(go.Scatter(
    y = data['Linear_Acceleration_y_(m/s^2)'],
    x = data.index,
    line =  dict(shape =  'spline' ),
    name = 'Signal with noise'
))

fig.add_trace(go.Scatter(
    y = data['y_filtered'],
    x = data.index,
    line =  dict(shape =  'spline' ),
    name = 'filtered signal'
))
fig.update_layout(
    title = "Processed Signal",
    xaxis_title = "Time (s)",
    yaxis_title = "Acc (m/s^2)"
)
fig.show()

In [ ]:
# Lowpass pada sinyal Inklinasi
tan_filtered = lowpass(data_inklinasi[0], cutoff,fs,order)
# Menyimpan hasil filter pada dataframe inklinasi
data_inklinasi['tan_filtered'] = tan_filtered

#Plotting hasil filter sinyal inklinasi
fig = go.Figure()
fig.add_trace(go.Scatter(
    y = data_inklinasi[0],
    x = data_inklinasi.index,
    line =  dict(shape =  'spline' ),
    name = 'Signal with noise'
))

fig.add_trace(go.Scatter(
    y = data_inklinasi['tan_filtered'],
    x = data_inklinasi.index,
    line =  dict(shape =  'spline' ),
    name = 'filtered signal'
))
fig.update_layout(
    title = "Processed Signal Inklinasi",
    xaxis_title = "Time (s)",
    yaxis_title = "Degree"
)
fig.show()


In [ ]:
def _boolrelextrema(data, comparator, axis=0, order=1, mode='clip'):
    if((int(order) != order) or (order < 1)): #Periksa apakah ORder setidaknya >= 1
        raise ValueError('Order must be an int >= 1')

    datalen = data.shape[axis] # Panjang data
    locs = np.arange(0, datalen) # Membuat Array [0, 1, 2, ... , n]

    results = np.ones(data.shape, dtype=bool) # Membuat Array Boolean samapai n
    main = data.take(locs, axis=axis, mode=mode) # Ambil semua data pada axis
    for shift in range(1, order + 1):
        plus = data.take(locs + shift, axis=axis, mode=mode) # Ambil data dengan offset locs + shift dengan mode clip sebagai default
        minus = data.take(locs - shift, axis=axis, mode=mode) # Ambil data dengan offset locs - shift dengan mode clip sebagai default
        results &= comparator(main, plus) # Membandingkan data main apakah lebih besar dari data plus dan melakukan clipping pada Result
        results &= comparator(main, minus) # Membandingkan data main apakah lebih besar dari data minus dan melakukan clipping pada Result
    return results #return Array of Boolean yang berisikan True jika dia sebiah puncak dan False Jika sebaliknya

def argrelextrema(data, comparator, axis=0, order=1, mode='clip'):
    results = _boolrelextrema(data, comparator,axis, order, mode) # return array of booelan yang berisikan indices puncak
    return np.nonzero(results) # Return indices yang tidak False

In [ ]:
# Mencari puncak menggunakan fungsi argrelexrema yang akan mengembalikan index puncak
y_peak = argrelextrema(data['y_filtered'].to_numpy(), np.greater_equal, 0,order)
# Plotting sinyal filter dengan marker puncak
fig = go.Figure()
fig.add_trace(go.Scatter(
    y = data['y_filtered'],
    x = data.index,
    line =  dict(shape =  'spline' ),
    name = 'filtered signal',
))


fig.add_trace(go.Scatter(
        mode='markers',
        y = data.iloc[y_peak].y_filtered,
        x = data.iloc[y_peak].index,
        marker=dict(
            color='LightSkyBlue',
            size=20,
            line=dict(
                color='MediumPurple',
                width=2
            )
        ),
        showlegend=False
    )
)
fig.update_layout(
    title = "Filtered Signal Direct",
    xaxis_title = "Time (s)",
    yaxis_title = "Acc (m/s^2)"
)
fig.show()

In [ ]:
# Mencari puncak menggunakan fungsi argrelexrema yang akan mengembalikan index puncak
inklinasi_peak = argrelextrema(data_inklinasi['tan_filtered'].to_numpy(), np.greater_equal)
# Plotting sinyal inklinasi filter dengan marker puncak
fig = go.Figure()
fig.add_trace(go.Scatter(
    y = data_inklinasi['tan_filtered'],
    x = data_inklinasi.index,
    line =  dict(shape =  'spline' ),
    name = 'filtered signal',
))


fig.add_trace(go.Scatter(
        mode='markers',
        y = data_inklinasi.iloc[inklinasi_peak].tan_filtered,
        x = data_inklinasi.iloc[inklinasi_peak].index,
        marker=dict(
            color='LightSkyBlue',
            size=20,
            line=dict(
                color='MediumPurple',
                width=2
            )
        ),
        showlegend=False
    )
)
fig.update_layout(
    title = "Filtered Signal Inklinasi",
    xaxis_title = "Time (s)",
    yaxis_title = "Degree"
)
fig.show()

In [ ]:
print("Jumlah peak dari metode Inklinasi adalah: ", len(inklinasi_peak[0]) , "nafas/menit")
print("Jumlah peak dari metode Direct adalah: ", len(y_peak[0]) ,"nafas/menit")

Jumlah peak dari metode Inklinasi adalah:  14 nafas/menit
Jumlah peak dari metode Direct adalah:  15 nafas/menit
